# 🧠 Iteracja 3: Chain-of-Thought (CoT) Prompting

**Czas:** ~45 minut  
**Poziom:** Średniozaawansowany

---

## Cel iteracji

Poprawić accuracy klasyfikacji przez wymuszenie na modelu **jawnego rozumowania krok po kroku** przed podaniem ostatecznej odpowiedzi.

---

## Teoria: Chain-of-Thought

**Chain-of-Thought (CoT)** to technika promptowania, w której instruujemy model, żeby przed podaniem odpowiedzi wypisał swój tok rozumowania.

### Dlaczego CoT działa?

Modele językowe generują tekst token po tokenie. Kiedy model "pisze swoje myśli", każdy wygenerowany token staje się kontekstem dla następnego - model dosłownie **"myśli głośno"** i może się korygować w trakcie.

```
Bez CoT:
  Recenzja → [model] → "bug"
  
Z CoT:
  Recenzja → [model] → "Recenzja mówi o crashach i FPS drops.  
                        Crasze to kategoria bug, FPS to performance.
                        Wniosek: bug, performance" → ["bug", "performance"]
```

### Rodzaje CoT

| Typ | Opis | Jak użyć |
|-----|------|----------|
| **Zero-shot CoT** | Magiczne zdanie w prompcie | `"Think step by step"` lub `"Przemyśl to krok po kroku"` |
| **Manual CoT** | Jawny format rozumowania | Pole `reasoning: str` przed `categories` w Pydantic |
| **Self-consistency** | Kilka przebiegów + głosowanie | Zaawansowane - nie dziś |

### Kluczowa zasada przy Pydantic + CoT:

> **Pole `reasoning` musi być PRZED `categories`** w modelu Pydantic.
> Model generuje pola w kolejności ich definicji.
> Jeśli `categories` jest pierwsze, model podaje kategorię PRZED rozumowaniem - efekt odwrotny!

```python
# ❌ ZŁE: categories przed reasoning
class Bad(BaseModel):
    categories: list[str]
    reasoning: str  # Model już podjął decyzję!

# ✅ DOBRE: reasoning przed categories  
class Good(BaseModel):
    reasoning: str   # Model "myśli" najpierw
    categories: list[str]  # Potem decyduje
```

## ⚙️ Setup - środowisko

Ta komórka wykrywa czy jesteś w **Google Colab** czy lokalnie, a następnie:
1. Klonuje repozytorium z GitHub (tylko Colab)
2. Instaluje wymagane biblioteki
3. Konfiguruje ścieżki importów

In [ ]:
import sys
import subprocess
from pathlib import Path

# Wykryj środowisko
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"{'🟡 Google Colab' if IN_COLAB else '💻 Lokalne środowisko'}")

# ============================================================
# KONFIGURACJA REPO (zmień URL na właściwe przed warsztatami)
# ============================================================
REPO_URL = "https://github.com/JSerek/techland-genai-workshop.git"
REPO_DIR = Path("/content/szkolenie_techland") if IN_COLAB else Path(".").resolve().parent
# ============================================================

if IN_COLAB:
    if not REPO_DIR.exists():
        print(f"📥 Klonuję repo...")
        subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
        print("✅ Repo sklonowane")
    else:
        print("🔄 Aktualizuję repo (git pull)...")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)

    # Dodaj repo do sys.path
    sys.path.insert(0, str(REPO_DIR))

    # Zainstaluj zależności
    print("📦 Instaluję biblioteki...")
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", str(REPO_DIR / "requirements.txt"), "-q"],
        check=True
    )
    print("✅ Biblioteki zainstalowane")
else:
    # Lokalnie: dodaj root projektu do sys.path
    sys.path.insert(0, str(REPO_DIR))
    print(f"📂 Ścieżka projektu: {REPO_DIR}")

print("\n✅ Setup zakończony")

In [ ]:
# ============================================================
# 🔑 Konfiguracja API
# W Google Colab: dodaj sekrety w menu po lewej (🔑 ikona)
#   Nazwy sekretów: VERTEX_AI_API_KEY, VERTEX_AI_BASE_URL, MODEL_NAME
#
# Lokalnie: ustaw zmienne środowiskowe lub wpisz wartości bezpośrednio
# ============================================================

if IN_COLAB:
    from google.colab import userdata
    API_KEY   = userdata.get("VERTEX_AI_API_KEY")
    BASE_URL  = userdata.get("VERTEX_AI_BASE_URL")
    MODEL_NAME = userdata.get("MODEL_NAME") or "google/gemini-2.5-flash-lite"
else:
    import os
    API_KEY   = os.environ.get("VERTEX_AI_API_KEY", "TODO: wklej API key")
    BASE_URL  = os.environ.get("VERTEX_AI_BASE_URL", "TODO: wklej endpoint URL")
    MODEL_NAME = os.environ.get("MODEL_NAME", "google/gemini-2.5-flash-lite")

# Walidacja
if not API_KEY or API_KEY == "TODO: wklej API key":
    print("❌ Brak API_KEY! Dodaj sekret 'VERTEX_AI_API_KEY' w Colab Secrets.")
elif not BASE_URL or BASE_URL == "TODO: wklej endpoint URL":
    print("❌ Brak BASE_URL! Dodaj sekret 'VERTEX_AI_BASE_URL' w Colab Secrets.")
else:
    print(f"✅ API skonfigurowane: {MODEL_NAME}")
    print(f"   Endpoint: {BASE_URL[:60]}...")

In [ ]:
# Konfiguracja klienta LLM - dane pobrane z Secrets w komórce powyżej
from src.utils.llm_client import create_client, LLMProvider

PROVIDER = LLMProvider.VERTEX_AI  # Zmień jeśli używasz innego providera

client = create_client(
    provider=PROVIDER,
    api_key=API_KEY,
    base_url=BASE_URL,
)
print(f"✅ Klient LLM gotowy: {PROVIDER.value} / {MODEL_NAME}")

In [ ]:
import json
import pandas as pd
from pathlib import Path
from pydantic import BaseModel, Field
from typing import Optional

from src.evaluation.evaluator import evaluate_trial, compare_trials, MatchStrategy


# Kategorie tematyczne
CATEGORIES = [
    "gameplay",       # Mechaniki rozgrywki, walka, parkour, sterowanie, balans
    "story",          # Fabuła, postacie, dialogi, scenariusz
    "bugs",           # Crashe, glitche, błędy, problemy techniczne
    "performance",    # FPS, lag, stuttering, optymalizacja ładowania
    "content",        # Ilość contentu, długość gry, powtarzalność
    "graphics",       # Grafika, wizualia, animacje
    "audio",          # Muzyka, dźwięki, voice acting
    "price",          # Cena, wartość za pieniądze, DLC
]
categories_str = ', '.join(f'"{{c}}"' for c in CATEGORIES)

print('✅ Biblioteki załadowane')
print(f'Kategorie ({len(CATEGORIES)}): {categories_str}')


In [ ]:
DATA_DIR = REPO_DIR / 'data'

# Wczytaj golden dataset
golden_path = DATA_DIR / 'evaluation' / 'golden_dataset.json'
with open(golden_path) as f:
    golden_data = json.load(f)

golden_texts  = [item['review_text'] for item in golden_data]
golden_labels = [item['labels']      for item in golden_data]

print(f'✅ Golden dataset: {len(golden_data)} recenzji')
print(f'   Kategorie: {sorted(set(l for item in golden_data for l in item["labels"]))}')

## 📋 Golden Dataset

Poniżej 100 recenzji z przypisanymi kategoriami — to nasz punkt odniesienia do ewaluacji.
Model będzie klasyfikował te same recenzje, a my sprawdzimy czy jego wyniki zgadzają się z etykietami.

| Kategoria | Opis |
|-----------|------|
| `gameplay` | Mechaniki rozgrywki, walka, parkour, sterowanie, balans |
| `story` | Fabuła, postacie, dialogi, scenariusz |
| `bugs` | Crashe, glitche, błędy, problemy techniczne |
| `performance` | FPS, lag, stuttering, optymalizacja |
| `content` | Ilość contentu, długość gry, powtarzalność |
| `graphics` | Grafika, wizualia, animacje |
| `audio` | Muzyka, dźwięki, voice acting |
| `price` | Cena, wartość za pieniądze, DLC |

In [ ]:
from IPython.display import display, HTML

def show_golden_dataset(data: list[dict], max_text: int = 120) -> None:
    """Wyświetla golden dataset jako sformatowaną tabelę HTML."""
    rows = []
    for item in data:
        text = item['review_text']
        if len(text) > max_text:
            text = text[:max_text] + '...'
        # Każda etykieta jako kolorowy badge
        COLORS = {
            'gameplay':    '#4C72B0', 'story':       '#DD8452',
            'bugs':        '#55A868', 'performance': '#C44E52',
            'content':     '#8172B2', 'graphics':    '#937860',
            'audio':       '#DA8BC3', 'price':       '#8C8C8C',
        }
        badges = ' '.join(
            f'<span style="background:{COLORS.get(l, "#888")};color:white;'
            f'padding:2px 8px;border-radius:10px;font-size:11px;">{l}</span>'
            for l in item['labels']
        )
        rows.append(f'<tr><td style="text-align:center;color:#888;">{item["id"]}</td>'
                    f'<td style="font-size:12px;">{text}</td>'
                    f'<td>{badges}</td></tr>')

    html = f'''
    <style>
        .golden-table {{ border-collapse: collapse; width: 100%; font-family: sans-serif; }}
        .golden-table th {{ background: #2d2d2d; color: white; padding: 8px 12px; text-align: left; }}
        .golden-table td {{ padding: 6px 12px; border-bottom: 1px solid #eee; vertical-align: top; }}
        .golden-table tr:hover td {{ background: #f9f9f9; }}
    </style>
    <table class="golden-table">
        <thead><tr>
            <th style="width:40px">#</th>
            <th>Recenzja</th>
            <th style="width:200px">Kategorie</th>
        </tr></thead>
        <tbody>{''.join(rows)}</tbody>
    </table>
    '''
    display(HTML(html))

print(f'Golden dataset — {len(golden_data)} recenzji:')
show_golden_dataset(golden_data)

## 🔬 Eksperyment: Zero-shot CoT

Najprostsza forma CoT - jedno zdanie w prompcie.

In [ ]:
class ReviewClassification(BaseModel):
    """Klasyfikacja bez CoT (baseline z Iteracji 2)."""
    categories: list[str] = Field(description=f"Kategorie: {CATEGORIES}")


class ReviewClassificationCoT(BaseModel):
    """Klasyfikacja z Chain-of-Thought - reasoning PRZED categories."""

    reasoning: str = Field(
        description=(
            "Krok po kroku analiza recenzji: "
            "1) Co konkretnie opisuje recenzja? "
            "2) Które fragmenty wskazują na poszczególne kategorie? "
            "3) Ostateczne uzasadnienie wyboru kategorii."
        )
    )
    categories: list[str] = Field(
        description=f"Finalna lista kategorii po analizie. Dozwolone: {CATEGORIES}"
    )


print("✅ Modele zdefiniowane")
print(f"ReviewClassification pola: {list(ReviewClassification.model_fields.keys())}")
print(f"ReviewClassificationCoT pola: {list(ReviewClassificationCoT.model_fields.keys())}")

In [ ]:
# Systemy promptów: bez CoT vs z CoT

SYSTEM_PROMPT_BASELINE = f"""Klasyfikuj recenzję gry do kategorii tematycznych.
Dostępne kategorie: {categories_str}
Przypisz jedną lub więcej pasujących kategorii."""

SYSTEM_PROMPT_COT = f"""Jesteś ekspertem od analizy recenzji gier komputerowych.
Klasyfikujesz recenzje do kategorii tematycznych.

Dostępne kategorie: {categories_str}

Twój proces:
1. Przeczytaj recenzję uważnie
2. Zidentyfikuj konkretne elementy/problemy opisane w recenzji
3. Dla każdego elementu - określ odpowiadającą kategorię
4. Podaj finalną listę kategorii

Przemyśl to krok po kroku przed podaniem ostatecznej odpowiedzi."""

USER_PROMPT = "Recenzja do sklasyfikowania: {review_text}"

In [ ]:
# Demo: porównanie baseline vs CoT na jednej recenzji
test_review = review_texts[0]
print(f"Recenzja: '{test_review[:300]}'\n")

# Baseline (bez CoT)
result_baseline = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_BASELINE},
        {"role": "user", "content": USER_PROMPT.format(review_text=test_review)},
    ],
    response_model=ReviewClassification,
    temperature=0.0,
)

# Z CoT
result_cot = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_COT},
        {"role": "user", "content": USER_PROMPT.format(review_text=test_review)},
    ],
    response_model=ReviewClassificationCoT,
    temperature=0.0,
)

print("=" * 50)
print("BASELINE (bez CoT):")
print(f"  Kategorie: {result_baseline.categories}")

print("\n" + "=" * 50)
print("Z CHAIN-OF-THOUGHT:")
print(f"  Rozumowanie: {result_cot.reasoning}")
print(f"  Kategorie: {result_cot.categories}")

## 🎯 Ćwiczenie: Zoptymalizuj CoT prompt

### Zadanie:
Napisz własny system prompt z Chain-of-Thought i sprawdź czy poprawia accuracy.

**Eksperymenty do przeprowadzenia:**
1. Zmień język promptu (EN vs PL)
2. Dodaj przykłady toku rozumowania (manual CoT)
3. Zmień sformułowanie kroków analizy
4. Dodaj instrukcję sprawdzenia własnej odpowiedzi: `"Review your answer and correct any mistakes"`
5. Eksperymentuj z temperaturą (0.0 vs 0.3 vs 0.7)

In [ ]:
# ============================================================
# 🖊️  ĆWICZENIE: Twój CoT prompt
# ============================================================

MY_COT_SYSTEM_PROMPT = """
# TODO: Napisz swój system prompt z Chain-of-Thought
"""

# Opcjonalnie: własny Pydantic model z innym polem reasoning
# class MyCoTModel(BaseModel):
#     ...

print("Uzupełnij MY_COT_SYSTEM_PROMPT i przetestuj!")

## 📊 Porównanie: Iteracja 2 vs Iteracja 3

In [ ]:
from tqdm.notebook import tqdm

texts_to_classify = golden_texts if golden_texts else review_texts[:10]

# Klasyfikacja CoT
print("Klasyfikuję z Chain-of-Thought...")
predictions_cot = []
for text in tqdm(texts_to_classify):
    try:
        result = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_COT},
                {"role": "user", "content": USER_PROMPT.format(review_text=text)},
            ],
            response_model=ReviewClassificationCoT,
            temperature=0.0,
        )
        predictions_cot.append(result.categories)
    except Exception as e:
        print(f"Błąd: {e}")
        predictions_cot.append([])

print("✅ Gotowe")

In [ ]:
if golden_labels:
    trial3 = evaluate_trial(
        trial_name=f"{MODEL_NAME} + Chain-of-Thought",
        model=MODEL_NAME,
        prompt_variant="CoT",
        predictions=predictions_cot,
        expected=golden_labels,
        review_texts=golden_texts,
        strategy=MatchStrategy.CONTAINS_ALL,
    )
    trial3.display()

    # Porównanie z Iteracją 2 (załaduj trial2 jeśli masz)
    # compare_trials([trial2, trial3])
    
    print(f"\n💾 Accuracy Iteracja 3: {trial3.summary.accuracy:.1%}")

---

## ✅ Podsumowanie Iteracji 3

**Czego się nauczyłeś:**
- Chain-of-Thought: dlaczego zmuszanie modelu do "myślenia" poprawia jakość
- Zero-shot CoT vs Manual CoT
- Dlaczego kolejność pól w Pydantic ma znaczenie (reasoning → categories)
- Analiza błędów: które recenzje model klasyfikuje źle i dlaczego

**Kluczowa zasada:**  
> Dla trudnych zadań klasyfikacyjnych zawsze dodaj `reasoning: str` jako pierwsze pole w modelu.

**Następna iteracja:**  
Few-shot learning - pokażemy modelowi przykłady poprawnych klasyfikacji.